# HA4


1\. Deploy the ERC20 token smart contract with any reasonable parameters (like name, symbol) and allocate (mint) assets to it.

We'll be using a local hardhat instance to deploy our contracts.  
Using npm, we install hardhat: npm install --save-dev hardhat  
We initialize a new hardhat project via: npx hardhat init. We'll choose to use the template project for simplicity.
Then, using npx, we run hardhat: npx hardhat node  
We now have a locally running RPC server at http://127.0.0.1:8545/.  
  
We then want to create and mint a new ERC20 token, like in the 2nd home assignment, so i'll be less verbose here.

In [7]:
from web3 import Web3 
import json

In [1]:
# Creating a minimal ERC20 token.
"""
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@openzeppelin/contracts/token/ERC20/ERC20.sol";

contract MyToken is ERC20 {
    constructor(
        string memory _name,
        string memory _symbol,
        uint256 _initialSupply
    ) ERC20(_name, _symbol) {
        // Mint initial supply to the contract deployer
        _mint(msg.sender, _initialSupply);
    }

    // Allows to additionally mint tokens, if needed
    function mint(address _to, uint256 _amount) external {
        _mint(_to, _amount);
    }
}
"""
# This is solidity code for the contract in ./contracts/MyToken.sol


'\n// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\nimport "@openzeppelin/contracts/token/ERC20/ERC20.sol";\n\ncontract MyToken is ERC20 {\n    constructor(\n        string memory _name,\n        string memory _symbol,\n        uint256 _initialSupply\n    ) ERC20(_name, _symbol) {\n        // Mint initial supply to the contract deployer\n        _mint(msg.sender, _initialSupply);\n    }\n\n    // Allows to additionally mint tokens, if needed\n    function mint(address _to, uint256 _amount) external {\n        _mint(_to, _amount);\n    }\n}\n'

In [5]:
# We then want to compile the contract. We'll use hardhat compile, since we already have it.
# We install the npm @openzeppelin/contracts package, which our contract imports using npm: npm install @openzeppelin/contracts
# We then compile the contract via: hardhat compile
# The results are located in ./artifacts. We only need ABI and bytecode in ./artifacts/contracts/MyToken.sol/MyToken.json
with open("./artifacts/contracts/MyToken.sol/MyToken.json", "r") as mytoken_json:
    mytoken_json = json.load(mytoken_json)
    mytoken_abi = mytoken_json["abi"]
    mytoken_bytecode = mytoken_json["bytecode"]

In [15]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
chain_id = 31337  # hardhat's default chain id
# We get accounts info on npx hardhat node startup.
private_key = "0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"  

# Derive the account from the private key
account = w3.eth.account.from_key(private_key)
deploy_address = account.address
print(f"Deploying from address: {deploy_address}")


# We then do the usual stuff: deploy the contract, mint tokens.
mytoken = w3.eth.contract(abi=mytoken_abi, bytecode=mytoken_bytecode)
initial_supply = w3.to_wei(1_000_000, 'ether')
construct_txn = mytoken.constructor(
    "LolToken",       
    "LOL",           
    initial_supply 
).build_transaction(
    {
        "from": deploy_address,
        "nonce": w3.eth.get_transaction_count(deploy_address),
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id,
    }
)

# Sign and send
signed = account.sign_transaction(construct_txn)
tx_hash = w3.eth.send_raw_transaction(signed.raw_transaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("Contract deployed at address:", tx_receipt.contractAddress)

# Create contract instance at the deployed address
mytoken_contract = w3.eth.contract(
    address=tx_receipt.contractAddress, abi=mytoken_abi
)


# Let's mint 5000 tokens to the deployer
mint_amount = w3.to_wei(5000, 'ether')
mint_txn = mytoken_contract.functions.mint(deploy_address, mint_amount).build_transaction(
    {
        "from": deploy_address,
        "nonce": w3.eth.get_transaction_count(deploy_address),
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id,
    }
)

signed_mint_txn = account.sign_transaction(mint_txn)
mint_tx_hash = w3.eth.send_raw_transaction(signed_mint_txn.raw_transaction)
mint_tx_receipt = w3.eth.wait_for_transaction_receipt(mint_tx_hash)
print("Mint transaction receipt:", mint_tx_receipt.transactionHash.hex())

# Check deployer's balance
deployer_balance = mytoken_contract.functions.balanceOf(deploy_address).call()
print(f"Deployer's token balance: {deployer_balance} (raw units)")

Deploying from address: 0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266
Contract deployed at address: 0xa513E6E4b8f2a923D98304ec87F64353C4D5C853
Mint transaction receipt: 34c8d9e7bdbde3d1a4dbfc20b90a05c18c86cde13cadd3c541a7ca2b585ee06f
Deployer's token balance: 1005000000000000000000000 (raw units)


2\. Write a test script to  
a. Declare three addresses: Sender, Receiver_1, and Receiver_2. Note: The sender is also a contract deployer  
b. Send some tokens to the Receiver_1 and check the expected balances. For example, if the Sender mints 10,000 tokens and sends 100 to the receiver, check that the receiver has 100 and the sender has remaining.  
c. Repeat step 2.b, but this time the receiver is Receiver_2 (any amount)  

In [ ]:
# Sine the task description doesn't require us to use the hardhat network, we'll simply use pytest.
# See solution in ./test/test_erc20.py